In [1]:
%matplotlib inline

In [2]:
from __future__ import print_function, division
import pandas
import src
import os, os.path
from operator import itemgetter
if not os.path.exists('data'):
    os.chdir('..')

INFO:gensim.corpora.sharded_corpus:Could not import Theano, will use standard float for default ShardedCorpus dtype.
2015-12-13 13:16:18 twoism gensim.corpora.sharded_corpus[4080] INFO Could not import Theano, will use standard float for default ShardedCorpus dtype.
INFO:summa.preprocessing.cleaner:'pattern' package not found; tag filters are not available for English
2015-12-13 13:16:18 twoism summa.preprocessing.cleaner[4080] INFO 'pattern' package not found; tag filters are not available for English


In [3]:
def best_pair(s, t):
    s = reversed(sorted(enumerate(s), key=itemgetter(1)))
    t = reversed(sorted(enumerate(t), key=itemgetter(1)))
    for (i, item_i), (j, item_j) in zip(s, t):
        if i == j:
            return i

In [4]:
projects = src.common.load_projects(dict(level='file'))
projects

[Project(name='tika', printable_name='tika v1.8', version='v1.8', ref='refs/tags/1.8', data_path='data/tika/', full_path='data/tika/v1.8/', src_path='data/tika/v1.8/src/', level='file'),
 Project(name='pig', printable_name='pig v0.14.0', version='v0.14.0', ref='refs/tags/release-0.14.0', data_path='data/pig/', full_path='data/pig/v0.14.0/', src_path='data/pig/v0.14.0/src/', level='file'),
 Project(name='bookkeeper', printable_name='bookkeeper v4.3.0', version='v4.3.0', ref='refs/tags/release-4.3.0', data_path='data/bookkeeper/', full_path='data/bookkeeper/v4.3.0/', src_path='data/bookkeeper/v4.3.0/src/', level='file'),
 Project(name='openjpa', printable_name='openjpa v2.3.0', version='v2.3.0', ref='refs/tags/2.3.0', data_path='data/openjpa/', full_path='data/openjpa/v2.3.0/', src_path='data/openjpa/v2.3.0/src/', level='file'),
 Project(name='mahout', printable_name='mahout v0.10.0', version='v0.10.0', ref='refs/tags/mahout-0.10', data_path='data/mahout/', full_path='data/mahout/v0.10.0

In [5]:
names = {'model': {'score': 'score',
                   'model_base_alpha': 'alpha',
                   'model_base_eta': 'eta',
                   'num_topics': 'K'
                  },
         'corpus': {'score': 'score',
                    'changeset_include_additions': 'Additions',
                    'changeset_include_context': 'Context',
                    'changeset_include_message': 'Message',
                    'changeset_include_removals': 'Removals',
                    },
        }
exps = ['triage', 'feature_location']
table_headers = {
    'model': ['K', 'alpha', 'eta', 'Feature Location', 'Triage'],
    'corpus': ['Additions', 'Removals', 'Context', 'Message', 'Feature Location', 'Triage']
}
formatters = {
    'Feature Location': lambda x: r"{\bf %.4f}" % x if x == max(main_df["Feature Location"]) else "%.4f" % x,
    'Triage': lambda x: r"{\bf %.4f}" % x if x == max(main_df["Triage"]) else "%.4f" % x,
}

full_tex = r"""
\begin{table}
\begin{spacing}{1.2}
\centering
\caption{MRR values of %s %s construction sweep}
\label{table:%s}
\vspace{0.2em}
%s
\end{spacing}
\end{table}
"""

tex_dir = os.path.expanduser("~/git/dissertation/tables")
for project in projects:
    for rq in names.keys():
        names[rq]['score'] = 'score'
        main_df = pandas.DataFrame(columns=names[rq])
        for exp in exps:
            path = os.path.join(project.full_path, 'optimized-%s-changeset-%s.csv' % (rq, exp))
            exp = ' '.join(exp.title().split('_'))
            names[rq]['score'] = exp
            exp_df = pandas.read_csv(path)
            exp_df = exp_df.rename(columns=names[rq])
            if len(main_df):
                main_df = main_df.merge(exp_df)
            else:
                main_df = exp_df
        
        # filter out uninteresting rows, like there was no corpus
        main_df = main_df[(main_df["Feature Location"] != 0) | (main_df["Triage"] != 0)]
        if rq == "model":
            main_df = main_df.sort(["K", "alpha", "eta"])
            
        label = "%s_%s_sweep" % (project.name, rq)
        op = os.path.join(tex_dir, label + ".tex")
        
        if len(main_df) > 24:
            tex = r"\parbox{.45\linewidth}{\centering %s} \hfill \parbox{.45\linewidth}{\centering %s}"
            mid = len(main_df)//2
            tex = tex % (main_df[:mid].to_latex(index=False,
                                                escape=False, # needed so it doesn't screw up formatters
                                                formatters=formatters,
                                                columns=table_headers[rq]),
                         main_df[mid:].to_latex(index=False,
                                                escape=False, # needed so it doesn't screw up formatters
                                                formatters=formatters,
                                                columns=table_headers[rq]))
        else:
            tex = main_df.to_latex(index=False,
                                   escape=False, # needed so it doesn't screw up formatters
                                   formatters=formatters,
                                   columns=table_headers[rq],)
            
        print(best_pair(main_df["Triage"], main_df["Feature Location"]))
        
        # and now the lazy
        this_full_tex = full_tex % (project.printable_name.capitalize().replace("keeper", "Keeper").replace("Openjpa", "OpenJPA"), rq, label, tex)
        this_full_tex = this_full_tex.replace(" alpha ", r" $\alpha$ ")
        this_full_tex = this_full_tex.replace(" eta ", r" $\eta$ ")
        this_full_tex = this_full_tex.replace(" Feature Location ", " FLT ")
        this_full_tex = this_full_tex.replace(" Triage ", " DIT ")
        this_full_tex = this_full_tex.replace(r"\begin{tabular}{rllrr}", r"\begin{tabular}{rll|rr}")
        this_full_tex = this_full_tex.replace(r"\begin{tabular}{llllrr}", r"\begin{tabular}{llll|rr}")
        print("Writing to: %s\n%s\n" % (op, this_full_tex))
        with open(op, 'wt') as f:
            f.write(this_full_tex)

31
Writing to: /home/cscorley/git/dissertation/tables/tika_model_sweep.tex

\begin{table}
\begin{spacing}{1.2}
\centering
\caption{MRR values of Tika v1.8 model construction sweep}
\label{table:tika_model_sweep}
\vspace{0.2em}
\parbox{.45\linewidth}{\centering \begin{tabular}{rll|rr}
\toprule
   K & $\alpha$ &   $\eta$ & FLT & DIT \\
\midrule
 100 &     1 &     1 &           0.3619 & 0.3225 \\
 100 &     1 &     2 &           0.2483 & 0.3097 \\
 100 &     1 &     5 &           0.3056 & 0.2707 \\
 100 &     1 &  auto &           0.3354 & 0.2926 \\
 100 &     2 &     1 &     {\bf 0.4189} & 0.2766 \\
 100 &     2 &     2 &           0.2798 & 0.3082 \\
 100 &     2 &     5 &           0.3333 & 0.2563 \\
 100 &     2 &  auto &           0.3778 & 0.3179 \\
 100 &     5 &     1 &           0.2876 & 0.2303 \\
 100 &     5 &     2 &           0.3081 & 0.3001 \\
 100 &     5 &     5 &           0.1587 & 0.2094 \\
 100 &     5 &  auto &           0.3772 & 0.3020 \\
 100 &  auto &     1 &         

In [6]:
main_df

,Triage,Additions,Context,Message,Removals,Feature Location
0,0.361693,True,True,True,True,0.398153
1,0.347776,True,True,False,True,0.341635
2,0.380661,True,False,True,True,0.432896
3,0.360878,True,False,False,True,0.328378
4,0.375769,True,True,True,False,0.431324
5,0.350388,True,True,False,False,0.350432
6,0.362675,True,False,True,False,0.418561
7,0.353607,True,False,False,False,0.351343
8,0.326706,False,True,True,True,0.375887
9,0.321279,False,True,False,True,0.294647
